In [1]:
#好像每次都要整体重新运行，不然就报错运行不了
#tensorboard --logdir=D:\Anaconda3\Tensorflow\logs
#可以一次执行test和train的
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('D:/Anaconda3/Tensorflow/MNIST_data',one_hot=True)

batch_size = 100
#n_batch好像没用到啊
n_batch = mnist.train.num_examples // batch_size

#收集参数的均值方差
def Variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.mean(var)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.squre(var-mean)))#标准差公式
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

def weight_variable(shape,name):
    initial = tf.truncated_normal(shape,stddev=0.1)#生成一个截断的正态分布
    return tf.Variable(initial,name)

def bias_variable(shape,name):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial,name)

def conv2d(x,W):
    #x input tensor of shape `[batch批次大小, in_height长, in_width宽, in_channels通道数（黑白为1，彩色为3]`
    #W filter / kernel 滤波器卷积核tensor of shape [filter_height, filter_width, in_channels, out_channels]
    #`strides[0] = strides[3] = 1`. strides[1]代表x方向的步长，strides[2]代表y方向的步长
    #padding: A `string` from: `"SAME在外补0", "VALID不补0"`
    #2d表示二维卷积操作
    return tf.nn.conv2d(x,W,[1,1,1,1],padding='SAME')

def max_pooling_2x2(x):
    #ksize [1,x,y,1]窗口大小
    #ksize[0,3]默认为1，[1，2]为窗口大小
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

with tf.name_scope('input'):
    x = tf.placeholder(tf.float32,[None,784],name='x_input')
    y = tf.placeholder(tf.float32,[None,10],name = 'y_input')
    with tf.name_scope('x_image'):
        #向量转换成图片形状
        #改变x的格式转为4D的向量[batch, in_height, in_width, in_channels]`
        x_image = tf.reshape(x,[-1,28,28,1],name='x_image')
        
#第一个卷积层        
with tf.name_scope('conv1'):
    with tf.name_scope('conv1_w'):
        #5*5的采样窗口，1/32表示输入/输出通道数
        conv1_w = weight_variable([5,5,1,32],name='conv1_w')
    with tf.name_scope('conv1_b'):
        #一个卷积核一个偏置
        conv1_b = bias_variable([32],name='conv1_b')
    #把x_image和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数    
    with tf.name_scope('conv2d_1'):
        conv2d_1 = conv2d(x_image,conv1_w) + conv1_b
    with tf.name_scope('relu'):
        h_conv_1 = tf.nn.relu(conv2d_1)
    with tf.name_scope('conv1_pool'):
        conv1_pool = max_pooling_2x2(h_conv_1)
        
#第二个卷积层        
with tf.name_scope('conv2'):
    with tf.name_scope('conv2_w'):
        conv2_w = weight_variable([5,5,32,64],name='conv2_w')
    with tf.name_scope('conv2_b'):
        conv2_b = bias_variable([64],name='conv2_b')
    with tf.name_scope('conv2d_2'):
        conv2d_2 = conv2d(conv1_pool,conv2_w) + conv2_b
    with tf.name_scope('relu'):
        h_conv_2 = tf.nn.relu(conv2d_2)
    with tf.name_scope('conv2_pool'):
        conv2_pool = max_pooling_2x2(h_conv_2)

#28*28的图片第一次卷积后还是28*28，第一次池化后变为14*14
#第二次卷积后为14*14，第二次池化后变为了7*7
#进过上面操作后得到64张7*7的平面

#第一个全连接层        
with tf.name_scope('fc1'):
    with tf.name_scope('fc1_w'):
        #64个特征平面，每一层为7*7，即上一层神经元有7*7*64个
        fc1_w = weight_variable([7*7*64,1024],name='fc1_w')
    with tf.name_scope('fc1_b'):
        fc1_b = bias_variable([1024],name='fc1_b')
    #把池化层2的输出扁平化为1维
    with tf.name_scope('conv2_pool_flat'):
        conv2_pool_flat = tf.reshape(conv2_pool,[-1,7*7*64],name='conv2_pool_flat')
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(conv2_pool_flat,fc1_w) + fc1_b
    with tf.name_scope('relu'):
        h_fc1 = tf.nn.relu(wx_plus_b)
    with tf.name_scope('keep_prob'):
        keep_prob = tf.placeholder(tf.float32,name='keep_prob')
    with tf.name_scope('fc1_drop'):
        fc1_drop = tf.nn.dropout(h_fc1,keep_prob,name='fc1_drop')
        
#第二个全连接层    
with tf.name_scope('fc2'):
    with tf.name_scope('fc2_w'):
        fc2_w = weight_variable([1024,10],name='fc2_w')
    with tf.name_scope('fc2_b'):
        fc2_b = bias_variable([10],name='fc2_b')
    with tf.name_scope('wx_plus_b2'):
        wx_plus_b2 = tf.matmul(fc1_drop,fc2_w) + fc2_b
    with tf.name_scope('prediction'):
        prediction = tf.nn.softmax(wx_plus_b2)
        
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction),name='loss')
    tf.summary.scalar('loss',loss)
    
with tf.name_scope('train'):
    train = tf.train.AdamOptimizer(1e-4).minimize(loss)
    
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        #argmax返回一维张量中最大的值所在的位置
        #布尔型
        correct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
    with tf.name_scope('accuracy'):
        #tf.cast转换数据类型，转成浮点型
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)
#合并所有summary
merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #保存tensorboard
    train_writer = tf.summary.FileWriter('D:/Anaconda3/Tensorflow/logs/train',sess.graph)
    test_writer = tf.summary.FileWriter('D:/Anaconda3/Tensorflow/logs/test',sess.graph)

    for i in range(1001):
        batch_xs,batch_ys = mnist.train.next_batch(batch_size)
        sess.run(train,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        #记录训练集计算的参数
        summary = sess.run(merged,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
        train_writer.add_summary(summary,i)
        batch_xs,batch_ys =  mnist.test.next_batch(batch_size)
        #记录测试集计算的参数
        summary = sess.run(merged,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
        test_writer.add_summary(summary,i)
    
        if i%100 == 0:
            test_acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
            train_acc = sess.run(accuracy,feed_dict={x:mnist.train.images[:10000],y:mnist.train.labels[:10000],keep_prob:1.0})
            #+str(i)是为了好连接，也可以"Iter: %d"  %i
            print("Iter: " + str(i)+ ",Testing Accuracy: " + str(test_acc) + ",Training Accuracy:"+str(train_acc))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting D:/Anaconda3/Tensorflow/MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting D:/Anaconda3/Tensorflow/MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting D:/Anaconda3/Tensorflow/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting D:/Anaconda3/Tensorflow/MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Type is unsupp